In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
base_path = "/content/drive/My Drive/yl_tez"

In [ ]:
import os
if 'COLAB_GPU' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')
else:
    print("Yerelde Çalışıyor.")
    base_path = "."

In [ ]:
model_adi = "BestModelParameters"
model_yolu = os.path.join(base_path, model_adi)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_yolu)
model = AutoModelForCausalLM.from_pretrained(model_yolu)
model.to('cuda')
model.eval()

In [ ]:
def generate_text(full_prompt_text, tokenizer, model, device, max_length=100):
    # Girdi metnini tokenize et ve attention mask oluştur
    encoded_input = tokenizer.encode(full_prompt_text, return_tensors='pt', padding=True, truncation=True, max_length=max_length).to(device)

    # Metin üretimi
    output_sequences = model.generate(
        input_ids=encoded_input,
        max_length=max_length + 20,  # Cevap için ekstra uzunluk
        temperature=1.0,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        do_sample=True,
        num_return_sequences=1
    )
    generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
    return generated_text
def apply_chat_template(message):
    doctor_info = f"<DOCTOR_TITLE> Op. Dr. <SPECIALITY> kadin-hastaliklari-ve-dogum\n"
    user_message = f"<USER> {message}\n"
    assistant_message = f"<ASSISTANT>"
    return f"{doctor_info}\n{user_message}\n{assistant_message}"

def test_et(tokenizer, model, device):
    while True:
        # Kullanıcıdan soru girdisi alma
        prompt_text = input("Soru: ")
        if prompt_text == "exit":
            break
        # Tam soru metnini oluştur
        full_prompt_text = apply_chat_template(prompt_text)

        # Metin üretimi ve çıktının gösterilmesi
        generated_text = generate_text(full_prompt_text, tokenizer, model, device)
        print("Modelin ürettiği cevap:", generated_text)

In [ ]:
test_et(tokenizer, model, 'cuda')